# Function Call - Google Custom Search API 版

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from googleapiclient.discovery import build

# 環境変数の取得
load_dotenv()
# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
model_name = "gpt-3.5-turbo-0125"

google_cse_id = os.environ['GOOGLE_CSE_ID']
google_api_key = os.environ['GOOGLE_API_KEY']

# Google検索API

In [2]:
import json

# ニュースを返す関数の作成
def get_search_result(keyword, when, unit="d"):
    service = build("customsearch", "v1", developerKey=google_api_key)
    result = service.cse().list(q=keyword, dateRestrict=f"{when}{unit}", cx=google_cse_id).execute()

    # 検索結果のトップ3を配列に取得
    result_list = []
    for i, item in enumerate(result["items"]):
        result_list.append({"title": f'{item["title"]} {item["snippet"]}'})
        if i>=2:
            break

    return json.dumps({"result": result_list})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベント", 1, "m")
ret2 = json.loads(ret)
for n in ret2["result"]:
    print("-"*20)
    print(n["title"])

--------------------
東京キャラクターストリート特設 ｜ 東京駅一番街 東京キャラクターストリート内の「いちばんプラザ」やワゴンショップなど期間限定ショップやイベントも開催中！ いつも楽しいワクワクを集めてみなさまのお越しを ...
--------------------
東京駅周辺・丸の内でおすすめのイベント｜レッツエンジョイ東京 東京駅周辺・丸の内でおすすめのイベント · 東京都 中央区. YAESU BIG HOLIDAY～親子で楽しむ11日間～ · 東京都 千代田区. 丸の内ハウス 17TH ANNIVERSARY · 東京都 中央区.
--------------------
イベント&ニュース ｜ 東京駅一番街 イベント&ニュース · ウルトラマンワールドM78 スペシャル缶バッジプレゼントキャペーン. EVENT · 大谷翔平＆デコピンPOPUP SHOP. EVENT · MINECRAFT POP UP STORE · 山 ...


In [4]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [5]:
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

In [6]:
response

ChatCompletion(id='chatcmpl-9I4HlxDk4S6ui2g0nVZ3RyZVagmnv', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_80IPllaHH2KkpxmxJfJZ9vdQ', function=Function(arguments='{"keyword":"東京駅のイベント","when":1,"unit":"m"}', name='get_search_result'), type='function')]))], created=1714093405, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=33, prompt_tokens=135, total_tokens=168))

In [7]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"東京駅のイベント","when":1,"unit":"m"}'

In [8]:
# モデルがツール呼出と判断した
if response.choices[0].message.tool_calls is not None:
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    print(f"関数名：{function_name}")
    # 引数の取得
    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    print(f"関数の実行結果：{function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0].message.content.strip())

関数名：get_search_result
引数：{'keyword': '東京駅のイベント', 'when': 1, 'unit': 'm'}
関数の実行結果：{"result": [{"title": "\u6771\u4eac\u30ad\u30e3\u30e9\u30af\u30bf\u30fc\u30b9\u30c8\u30ea\u30fc\u30c8\u7279\u8a2d \uff5c \u6771\u4eac\u99c5\u4e00\u756a\u8857 \u6771\u4eac\u30ad\u30e3\u30e9\u30af\u30bf\u30fc\u30b9\u30c8\u30ea\u30fc\u30c8\u5185\u306e\u300c\u3044\u3061\u3070\u3093\u30d7\u30e9\u30b6\u300d\u3084\u30ef\u30b4\u30f3\u30b7\u30e7\u30c3\u30d7\u306a\u3069\u671f\u9593\u9650\u5b9a\u30b7\u30e7\u30c3\u30d7\u3084\u30a4\u30d9\u30f3\u30c8\u3082\u958b\u50ac\u4e2d\uff01 \u3044\u3064\u3082\u697d\u3057\u3044\u30ef\u30af\u30ef\u30af\u3092\u96c6\u3081\u3066\u307f\u306a\u3055\u307e\u306e\u304a\u8d8a\u3057\u3092\u00a0..."}, {"title": "\u6771\u4eac\u99c5\u5468\u8fba\u30fb\u4e38\u306e\u5185\u3067\u304a\u3059\u3059\u3081\u306e\u30a4\u30d9\u30f3\u30c8\uff5c\u30ec\u30c3\u30c4\u30a8\u30f3\u30b8\u30e7\u30a4\u6771\u4eac \u6771\u4eac\u99c5\u5468\u8fba\u30fb\u4e38\u306e\u5185\u3067\u304a\u3059\u3059\u3081\u306e\u30a4\u30d9\u3

# 複数のFunctionの切り替え

In [9]:
import json
import requests
from bs4 import BeautifulSoup

# ニュースを検索する関数
def get_news(keyword, when, unit="d"):
    # WebサイトのURLを指定
    url = f'https://news.google.com/search?q={keyword} when:{when}{unit}&hl=ja&gl=JP&ceid=JP:ja'

    # Requestsを利用してWebページを取得する
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article div div div a')

    # ニュースのトップ3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i>=2:
            break

    return json.dumps({"news": news})

In [10]:
# テスト用コード
ret = get_news("お祭り", 5)
ret2 = json.loads(ret)
for n in ret2["news"]:
    print("-"*20)
    print(n["title"])

--------------------
弟は「型抜き」で散財。一方、姉は…？お祭りから見る姉弟の違いがおもしろい！“昭和の子どもあるある”を描くノスタルジック漫画
--------------------
リオ、ベネチア…「行ってみたい海外のお祭りランキング」TOP10 | サライ.jp｜小学館の雑誌『サライ』公式サイト
--------------------
お祭りを楽しむ“ゼイユ＆スグリ”のプレイマットも！『ポケカ』4月26日発売の「周辺グッズ」まとめ


In [11]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    }),
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_news",
            "description": "指定したキーワードのニュースを取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [12]:
# question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
question = "お祭りに関する5日以内のニュースを教えてください"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)
response

ChatCompletion(id='chatcmpl-9I4JQB1X25izHt93O4vkMncQkt7pf', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aqNnA8oVQjpTtL13stFEPUCD', function=Function(arguments='{"keyword":"お祭り","when":5,"unit":"d"}', name='get_news'), type='function')]))], created=1714093508, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=25, prompt_tokens=187, total_tokens=212))

In [13]:
response

ChatCompletion(id='chatcmpl-9I4JQB1X25izHt93O4vkMncQkt7pf', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aqNnA8oVQjpTtL13stFEPUCD', function=Function(arguments='{"keyword":"お祭り","when":5,"unit":"d"}', name='get_news'), type='function')]))], created=1714093508, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=25, prompt_tokens=187, total_tokens=212))

In [14]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"お祭り","when":5,"unit":"d"}'

In [15]:
# モデルがツール呼出と判断した
if response.choices[0].message.tool_calls is not None:
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    # 引数の取得
    arguments = json.loads(tool.function.arguments)

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    elif function_name == "get_news":
        function_response = get_news(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0].message.content.strip())

以下はお祭りに関連する最新のニュースです：

1. 「兄は『型抜き』で窃盗。一方、姉は…？お祭りから見る姉弟の違いがおもしろい！“昭和の子ども２人いる2人”を描くノスタルジック漫画
2. リオ、ベネチア…「行ってみたい海外のお祭りランキング」TOP10 | サライ.jp｜小学館の雑誌『サライ』公式サイト
3. お祭りを楽しむ“ハイ＆スグリ”のプレママコーデも！『ポケカ』4月26日発売の「周辺ガチボス」まとめ

詳細情報や興味がある記事があれば、検索して詳細を確認してみてください。
